# Numerical Captcha Reader using TensorFlow with Amazon SageMaker

## Introduction

This sample project demonstrates how to use Amazon SageMaker to build, train and deploy Tensorflow model.In this example numerical captcha of 4 digit length is used as input imaged and inference is done to read the input captcha digits. 

[Amazon SageMaker](https://aws.amazon.com/sagemaker/) is a fully managed machine learning service. With SageMaker, data scientists and developers can quickly and easily build and train machine learning models, and then directly deploy them into a production-ready hosted environment.You can use Amazon SageMaker to train and deploy a model using custom TensorFlow code. The SageMaker Python SDK TensorFlow estimators and models and the SageMaker open-source TensorFlow containers make writing a TensorFlow script and running it in SageMaker easier.

In this example, we will show how easily you can train a Machine Learning model in Amazon SageMaker using TensorFlow 2.X scripts with SageMaker Python SDK. In addition, this notebook demonstrates how to perform real time inference with the [SageMaker TensorFlow Serving container](https://github.com/aws/sagemaker-tensorflow-serving-container). 

In [ ]:
!pip install captcha

## Set up the environment

Here we set up the linkage and authentication to AWS services. In this notebook we only need the roles used to give learning and hosting access to your data. The Sagemaker SDK will use S3 defualt buckets when needed. If the get_execution_role does not return a role with the appropriate permissions, you'll need to specify an IAM role arn that does.

In [ ]:
import os
import time
import json
import base64
import numpy as np
import sagemaker
from PIL import Image
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow
import tensorflow as tf
from tensorflow import keras

sagemaker_session = sagemaker.Session()

role   = get_execution_role()
region = sagemaker_session.boto_session.region_name
bucket = sagemaker_session.default_bucket()
prefix = 'captcha/char-4-epoch-60'

print('using bucket %s'%bucket)

In [ ]:
s3train = 's3://{}/{}/train/'.format(bucket, prefix)

## Training

For TensorFlow versions 1.11 and later, the [Amazon SageMaker Python SDK](https://sagemaker.readthedocs.io/) supports script mode training scripts.Script mode is a training script format for TensorFlow that lets you execute any TensorFlow training script in SageMaker with minimal modification. The [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) handles transferring your script to a SageMaker training instance. On the training instance, SageMaker's native TensorFlow support sets up training-related environment variables and executes your training script. In this tutorial, we use the SageMaker Python SDK to launch a training job and deploy the trained model. The `sagemaker.tensorflow.TensorFlow` estimator handles locating the script mode container, uploading your script to a S3 location and creating a SageMaker training job.

In [ ]:
captcha_estimator = TensorFlow(entry_point='captcha-tf.py',
                             role=role,
                             train_instance_count=1,
                             train_instance_type='ml.p3.2xlarge',
                             framework_version='2.1.0',
                             py_version='py3')

To start a training job, we call `estimator.fit()`. When training starts, the TensorFlow container executes training script (in this case) captcha-tf.py, passing `hyperparameters` and `model_dir` from the estimator as script arguments. Because we didn't define either in this example, no hyperparameters are passed, and `model_dir` defaults to `s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>`, so the script execution is as follows:

```
/usr/bin/python3 captcha-tf.py --model_dir s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>
```
    
When training is complete, the training job will upload the saved model for TensorFlow serving.

In [ ]:
captcha_estimator.fit({'train':s3train})

## Deploy the trained model to an endpoint

The `deploy()` method creates a SageMaker model, which is then deployed to an endpoint to serve prediction requests in real time. We will use the TensorFlow Serving container for the endpoint. The TensorFlow Serving container is the default inference method for script mode. This serving container runs an implementation of a web server that is compatible with SageMaker hosting protocol.

In [ ]:
endpoint_name = 'captcha-tensorflow'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
end_point = captcha_estimator.deploy(initial_instance_count=1,instance_type='ml.m5.4xlarge',endpoint_name=endpoint_name)

## Inference

The formats of the input and the output data correspond directly to the request and response formats of the Predict method in the [TensorFlow Serving REST API](https://www.tensorflow.org/serving/api_rest). SageMaker's TensforFlow Serving endpoints can also accept additional input formats that are not part of the TensorFlow REST API, including the simplified JSON format, line-delimited JSON objects ("jsons" or "jsonlines"), and CSV data.

In this example we are using input images converted to numpy array as input, which will be serialized into the simplified JSON format. In addtion, TensorFlow serving can also process multiple items at once as you can see in the following code. You can find the complete documentation on how to make predictions against a TensorFlow serving SageMaker endpoint [here](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html#making-predictions-against-a-sagemaker-endpoint).

In [ ]:
def preprocess_input(image_path):
    if (os.path.exists(image_path)):
        originalImage = Image.open(image_path)
        image = np.asarray(originalImage) / 255.
        image = tf.expand_dims(image,0)
        input_data = {'instances': np.asarray(image).astype(float)}
        return input_data
    else:
        print('input does not exist!\n')
        return None

Let's download the sample test data and use that as input for inference.

In [ ]:
#!aws s3 cp s3://<DEFAULT_BUCKET>/<Test-Data-Prefix>/0456_e49f9157-0881-4adf-aea2-cc0602c3149e.png /tmp
image_path = "/tmp/0456_e49f9157-0881-4adf-aea2-cc0602c3149e.png"

input_data = preprocess_input(image_path)
result = end_point.predict(input_data)
predicted_class_idx = np.argmax(result['predictions'][0], axis=-1)
print(predicted_class_idx)

#!aws s3 cp s3://<DEFAULT_BUCKET>/<Test-Data-Prefix>/8670_ad997de8-8657-463e-858f-36042404c962.png /tmp
image_path = "/tmp/8670_ad997de8-8657-463e-858f-36042404c962.png"
input_data = preprocess_input(image_path)
result = end_point.predict(input_data)
predicted_class_idx = np.argmax(result['predictions'][0], axis=-1)
print(predicted_class_idx)

#!aws s3 cp s3://<DEFAULT_BUCKET>/<Test-Data-Prefix>/0324_16e096f7-c96a-44bd-91f4-c5c33b1ec5c7.png /tmp
image_path = "/tmp/0324_16e096f7-c96a-44bd-91f4-c5c33b1ec5c7.png"
input_data = preprocess_input(image_path)
result = end_point.predict(input_data)
predicted_class_idx = np.argmax(result['predictions'][0], axis=-1)
print(predicted_class_idx)

## Delete the endpoint

Let's delete the endpoint we just created to prevent incurring any extra costs.

In [ ]:
end_point.delete_endpoint()